In [ ]:
# 1. Установка необходимых библиотек
!pip install -q langchain chromadb langchain-chroma sentence-transformers transformers ipywidgets huggingface_hub

In [ ]:
# 2. Клонируем репозиторий с файлами
!git clone https://github.com/IvanPetrovich/LLM_RAG_GoogleColab
%cd LLM_RAG_GoogleColab

In [ ]:
# 3. Импорт библиотек
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# 4. Загрузка модели генерации текста
generator = pipeline(
    "text-generation",
    model="google/gemma-3",
    device=-1
)

In [ ]:
# 5. Функция для загрузки текстовых файлов
def load_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

upcoming_text = load_file("upcoming.txt")
released_text = load_file("released.txt")

In [ ]:
# 6. Разделение текста на документы
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
upcoming_docs = text_splitter.create_documents([upcoming_text])
released_docs = text_splitter.create_documents([released_text])

In [ ]:
# 7. Создание векторных хранилищ
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_upcoming = Chroma.from_documents(upcoming_docs, embeddings)
db_released = Chroma.from_documents(released_docs, embeddings)

In [ ]:
# 8. Функция для поиска и генерации ответа
def generate_answer(query, db):
    docs = db.similarity_search(query)
    context = "\n".join([doc.page_content for doc in docs])
    prompt = f"На основе следующего контекста, ответь на вопрос:\n{context}\nВопрос: {query}\nОтвет:"
    response = generator(prompt, max_length=500)
    return response[0]["generated_text"]

In [ ]:
# 9. Пример использования
query = "Какие ожидаются новинки в играх?"
answer = generate_answer(query, db_upcoming)
print(answer)